## Business Problem: 
Nobel Prize winner Donna Strickland's page was deleted from Wikipedia a few months before she received the Nobel prize. A Wikipedia editor claimed that Strickland did not have significant coverage in published sources [1].

## Objective:
Explore websites cited on pages nominated for deletion. What types of websites are cited on pages about women? 

### References:
[1] Bazely, Dawn. "Why Nobel winner Donna Strickland didn’t have a Wikipedia page." Washington Post. 8 Oct. 2018. https://www.washingtonpost.com/outlook/2018/10/08/why-nobel-winner-donna-strickland-didnt-have-wikipedia-page/

In [1]:
from __future__ import print_function
import boto3
from bs4 import BeautifulSoup
import csv
import datetime
import io
import os
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
import sys
from urllib.parse import urlparse
from src import data
from src import scraping
from src import tests

In [2]:
def extract_domain(url):
    parsed_uri = urlparse(url)
    result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    
    if result == "https://web.archive.org/":
        archive_pattern = r'https://web.archive.org/web/\d+/'
        url = re.sub(archive_pattern, "", url)
        
        parsed_uri = urlparse(url)
        result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    return result

In [3]:
bucket = 'afd-scraped'
processed_data_path = './data/processed/'

In [4]:
# intialize connection to S3 resources
s3 = boto3.resource('s3')
s3_client = boto3.client('s3', 'us-east-1')

In [5]:
pd.set_option('display.max_colwidth', -1)

# Combine daily_afd_analysis csvs into one master csv

In [6]:
# extract list of keys for indiv_afd_analysis files 
keys = data.get_keys_in_bucket(s3_client,bucket,'daily_afd_analysis')

In [7]:
# combine all indiv_afd_analysis into one csv
page_scrapeDate = pd.DataFrame()
for k in keys:
    df = data.read_csv_on_s3(s3_client,bucket,k)
    page_scrapeDate = data.append_dataframes(page_scrapeDate,df)

/Users/elenasmith/Documents/women_in_red/wir_aws_v2/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


## Review data frame of wiki pages by scrape date

In [8]:
page_scrapeDate['scrape_date'] = page_scrapeDate['key'].apply(lambda x: x.split('/')[1])
page_scrapeDate['scrape_date'] = pd.to_datetime(page_scrapeDate['scrape_date'])
page_scrapeDate['page_key'] = page_scrapeDate['key'].apply(lambda x: x.split("/")[2])

In [9]:
page_scrapeDate = page_scrapeDate[page_scrapeDate['recommended_gender'].isin(['female','male'])]
page_scrapeDate[0:3]

,count_he,count_nonbinary,count_she,count_they,gender_rule,is_person_by_card,is_person_by_category,key,recommended_gender,reference_urls,scrape_date,page_key
18,2,0,0,1,he_ct_greater_she,1.0,1.0,individual_afd_page/2019-07-05/Andrew_Videto.txt,male,NaN,2019-07-05,Andrew_Videto.txt
21,0,0,2,0,she_ct_greater_he,1.0,1.0,individual_afd_page/2019-07-05/Anissa_Kate.txt,female,NaN,2019-07-05,Anissa_Kate.txt
22,0,0,6,0,she_ct_greater_he,0.0,1.0,individual_afd_page/2019-07-05/Ann_E._Harrison.txt,female,NaN,2019-07-05,Ann_E._Harrison.txt


## Identify the most recent date on which pages were scraped

In [10]:
page_key___mostRecent = page_scrapeDate.groupby('page_key').agg({"scrape_date": lambda x: max(x)}).reset_index()

In [11]:
original = page_scrapeDate.shape[0]
page_scrapeDate = page_scrapeDate.merge(page_key___mostRecent,
                           on = "page_key",
                           suffixes= ['','_mostRecent'])
assert original == page_scrapeDate.shape[0]

In [12]:
page = page_scrapeDate[page_scrapeDate['scrape_date']==page_scrapeDate['scrape_date_mostRecent']]
page[0:3]

,count_he,count_nonbinary,count_she,count_they,gender_rule,is_person_by_card,is_person_by_category,key,recommended_gender,reference_urls,scrape_date,page_key,scrape_date_mostRecent
9,2,0,0,1,he_ct_greater_she,1.0,1.0,individual_afd_page/2019-07-14/Andrew_Videto.txt,male,NaN,2019-07-14,Andrew_Videto.txt,2019-07-14
18,0,0,2,0,she_ct_greater_he,1.0,1.0,individual_afd_page/2019-07-13/Anissa_Kate.txt,female,NaN,2019-07-13,Anissa_Kate.txt,2019-07-13
26,0,0,6,0,she_ct_greater_he,0.0,1.0,individual_afd_page/2019-07-12/Ann_E._Harrison.txt,female,NaN,2019-07-12,Ann_E._Harrison.txt,2019-07-12


In [13]:
tests.test_key(page,['page_key'])

In [14]:
page.shape

(837, 13)

## Extract domains of cited websites

In [15]:
page___hasReferenceURL = page[pd.isnull(page['reference_urls'])==False]
print(page___hasReferenceURL.shape)

(618, 13)


In [16]:
page___hasReferenceURL['reference_urls_list'] = page___hasReferenceURL['reference_urls'].apply(lambda x: str(x).replace("['","").\
                                                           replace("']","").split("', '"))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
page___referenceURL = data.explode_list_column(page___hasReferenceURL,'reference_urls_list')
page___referenceURL = page___referenceURL.drop("exploded_reference_urls_list_index",axis=1)

# drop pages without any references
page___referenceURL = page___referenceURL[ page___referenceURL['reference_urls_list_exploded'] != '[]']

page___referenceURL = page___referenceURL.drop_duplicates()
tests.test_key(page___referenceURL,['page_key','reference_urls_list_exploded'])

In [18]:
page___referenceURL['domain'] = page___referenceURL['reference_urls_list_exploded'].apply(lambda x: 
    extract_domain(x))

## Count the number of times each domain is cited by gender. 

In [19]:
domain___gender = page___referenceURL.groupby(['domain','recommended_gender']).agg({"page_key": 
    lambda x: x.nunique()}).reset_index().sort_values('page_key',ascending=False)

In [20]:
domain = domain___gender[domain___gender['recommended_gender']=='male'].merge(domain___gender[domain___gender['recommended_gender']=='female'],
                                                                    on = "domain",
                                                                    how="outer",
                                                                    suffixes=['_male','_female'])
domain['page_key_male'] = domain['page_key_male'].fillna(0)
domain['page_key_female'] = domain['page_key_female'].fillna(0)

In [21]:
domain['raw_gender_ratio'] = domain[['page_key_male','page_key_female']].apply(lambda x: 
                                                                           x[0]/x[1] if x[1] > 0 else np.NaN
                                                                           ,axis=1)


In [22]:
# use add-one smoothing 
len_unique_domains = len(domain['domain'].unique())
num_domain_occurences = sum(domain['page_key_male']) +  sum(domain['page_key_female'])

def laplace_smoothing(domain_ct_by_gender, num_domain_occurences, len_unique_domains):
    return (domain_ct_by_gender + 1)/ (num_domain_occurences + len_unique_domains)

domain['laplace_prob_male'] = domain['page_key_male'].apply(lambda x: laplace_smoothing(x, num_domain_occurences, 
                                                                                       len_unique_domains))
domain['laplace_prob_female'] = domain['page_key_female'].apply(lambda x: laplace_smoothing(x, num_domain_occurences, 
                                                                                       len_unique_domains))


In [23]:
domain['laplace_ratio_male_to_female'] = domain[['laplace_prob_male','laplace_prob_female']].apply(lambda x: 
                                                                                                   x[0] / x[1],axis=1)

## Review domains more likely to appear on a page about a male nominated for deletion

In [24]:
domain[['domain','laplace_ratio_male_to_female','laplace_prob_male','page_key_male',
        'page_key_female']].sort_values('laplace_ratio_male_to_female',ascending=False)[0:10]

,domain,laplace_ratio_male_to_female,laplace_prob_male,page_key_male,page_key_female
14,https://issuu.com/,10.0,0.001107,9.0,0.0
17,https://int.soccerway.com/,8.0,0.000886,7.0,0.0
23,https://www.latimes.com/,7.0,0.000775,6.0,0.0
30,https://soundcloud.com/,6.0,0.000664,5.0,0.0
26,https://www.chicagotribune.com/,6.0,0.000664,5.0,0.0
25,https://www.cnbc.com/,6.0,0.000664,5.0,0.0
34,https://www.bizjournals.com/,6.0,0.000664,5.0,0.0
46,https://www.discogs.com/,5.0,0.000554,4.0,0.0
47,https://www.thetimes.co.uk/,5.0,0.000554,4.0,0.0
48,https://www.dallasnews.com/,5.0,0.000554,4.0,0.0


In [25]:
# review domains more likely to appear on male pages that appeared on at least one female page
domain[domain['page_key_female'] > 0][['domain','laplace_ratio_male_to_female','laplace_prob_male','page_key_male',
        'page_key_female']].sort_values('laplace_ratio_male_to_female',ascending=False)[0:10]

,domain,laplace_ratio_male_to_female,laplace_prob_male,page_key_male,page_key_female
4,https://www.bloomberg.com/,4.500000,0.001993,17.0,3.0
9,https://www.facebook.com/,4.333333,0.001439,12.0,2.0
18,https://www.linkedin.com/,4.000000,0.000886,7.0,1.0
22,https://www.entrepreneur.com/,3.500000,0.000775,6.0,1.0
20,https://www.thehindu.com/,3.500000,0.000775,6.0,1.0
8,https://archive.is/,3.250000,0.001439,12.0,3.0
10,https://twitter.com/,3.250000,0.001439,12.0,3.0
33,https://techcrunch.com/,3.000000,0.000664,5.0,1.0
13,https://www.wsj.com/,3.000000,0.001328,11.0,3.0
29,http://articles.latimes.com/,3.000000,0.000664,5.0,1.0


## Review domains more likely to appear on a page about a female nominated for deletion

In [26]:
domain[['domain','laplace_ratio_male_to_female','laplace_prob_female','page_key_male',
        'page_key_female']].sort_values('laplace_ratio_male_to_female')[0:10]

,domain,laplace_ratio_male_to_female,laplace_prob_female,page_key_male,page_key_female
2629,http://www.iafd.com/,0.200000,0.000554,0.0,4.0
2630,https://archive.today/,0.200000,0.000554,0.0,4.0
2631,http://business.avn.com/,0.200000,0.000554,0.0,4.0
2634,https://www.msn.com/,0.250000,0.000443,0.0,3.0
2632,http://www.mtv.co.uk/,0.250000,0.000443,0.0,3.0
2633,http://www.xbiz.com/,0.250000,0.000443,0.0,3.0
2635,https://www.iwmbuzz.com/,0.250000,0.000443,0.0,3.0
2636,https://www.rollingstone.com/,0.250000,0.000443,0.0,3.0
2637,https://www.belfasttelegraph.co.uk/,0.250000,0.000443,0.0,3.0
2669,http://culture.affinitymagazine.us/,0.333333,0.000332,0.0,2.0


In [27]:
# review domains more likely to appear on female pages that appeared on at least one male page
domain[domain['page_key_male'] > 0][['domain','laplace_ratio_male_to_female','laplace_prob_female','page_key_male',
        'page_key_female']].sort_values('laplace_ratio_male_to_female')[0:10]

,domain,laplace_ratio_male_to_female,laplace_prob_female,page_key_male,page_key_female
66,http://www.bbc.co.uk/,0.400000,0.001107,3.0,9.0
1693,http://nymag.com/,0.400000,0.000554,1.0,4.0
1102,https://www.dnaindia.com/,0.500000,0.000443,1.0,3.0
929,https://www.vox.com/,0.500000,0.000443,1.0,3.0
1262,https://www.instagram.com/,0.500000,0.000443,1.0,3.0
1998,http://indiatoday.intoday.in/,0.500000,0.000443,1.0,3.0
2413,http://www.dnaindia.com/,0.500000,0.000443,1.0,3.0
84,http://www.highbeam.com/,0.571429,0.000775,3.0,6.0
217,https://www.pressreader.com/,0.600000,0.000554,2.0,4.0
249,https://people.com/,0.666667,0.000332,1.0,2.0


* BBC and New York Magazine are reputable sources. Why were pages citing these sources nominated for deletion?

In [28]:
page___referenceURL[page___referenceURL['domain']=='http://www.bbc.co.uk/'][['page_key','recommended_gender','reference_urls','scrape_date']].drop_duplicates()

page_key recommended_gender  \
1333  Shreedeep_Rayamajhi.txt                 male                
1895  Ace_and_Vis.txt                         male                
4043  Rudi_Dharmalingam.txt                   male                
4108  Maria_Fowler.txt                        female              
5309  Amy_Garcia.txt                          female              
5336  Julia_George.txt                        female              
6369  List_of_vegans.txt                      female              
6835  Sara_Thornton_(weather_forecaster).txt  female              
7077  Amanda_Harper.txt                       female              
7147  Kaddy_Lee-Preston.txt                   female              
7270  Natalie_Graham.txt                      female              
7334  Wendy_Hurrell.txt                       female              

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [29]:
page___referenceURL[page___referenceURL['domain']=='http://nymag.com/'][['page_key','recommended_gender','reference_urls','scrape_date']].drop_duplicates()

page_key recommended_gender  \
236   Milagros_Schmoll.txt  female              
238   Tia_Walker.txt        female              
927   Dan_Humphrey.txt      female              
2842  Peter_Borish.txt      male                
3587  Sydney_Ember.txt      female              

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           